In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

In [ ]:
def removeNullist(remove):
    for i in remove:
        nullList.remove(i)
    print(nullList)

In [ ]:

trainSet = pd.read_csv('트레인.csv')
testSet = pd.read_csv('테스트.csv')

In [ ]:
# print(trainSet.columns)
# print(trainSet.head())
print(trainSet.tail())
# print(testSet.columns)
# print(testSet.head())
print(testSet.head())

In [ ]:
trainSetId=trainSet["Id"]
testSetId=testSet["Id"]
testSet.drop(columns=["Id"],inplace=True)
trainSet.drop(columns=["Id"],inplace=True)
trainSetSize = len(trainSet)
testSetSize = len(testSet)


In [ ]:
plt.figure(figsize=(20,20))
corrmat = trainSet.corr()
sns.heatmap(corrmat,annot=True, annot_kws={"size": 10})

In [ ]:
corrmat =(abs(corrmat)<=0.01)
plt.figure(figsize=(10,10))
sns.heatmap(corrmat,annot=True, annot_kws={"size": 20})
#대부분 가격과 연관성이 있다. 0.01보다 적단건 10억짜리 집에 이 요소가 100%있어도 1000만원 정도란 의미다.
#10억 있는 사람이 1000만원 때문에 집을 안사진 않는다.

In [ ]:
#굳이 y로그까진 필요 없을 듯하다. 이정도로 상관관계 변수들이 많다면, 
#괜히 각 요소의 영향력만 줄어들지도....(예를 들어 수영장 넓이가 커도 영향력은 작아보일지도 모른다.)
trainAnswer = trainSet['SalePrice']

trainSet.drop(['SalePrice'], axis=1, inplace=True)


In [ ]:
# 같이 
Set = pd.concat((trainSet, testSet), axis=0)

In [ ]:
nullList=[]
for i in list(Set):
    if (Set[i].isnull().sum())>0:
        nullList.append(i)
print(nullList)

In [ ]:
#일단 쓸모없는 애들부터 없애자.
nuls = Set.isnull().sum() / len(Set )

# 결혼상대로 따지면, 부모가 아랍석유부자인 것들.
# 설령 가격에 영향을 끼쳐도 이런 특징을 가진 사람은 다른것에서도 특별하니 가려준다.
# 예를 들면, 수영장이 있으면 당연히 집이 넓고, 차고도 있다.
# (기호 시설의 영향력은 줄어든다고 볼수 있다. 다음부터 생각해볼것) 
# 팬스는 20프로는 있으니 넘어가줄수 있었지만....솔직히 팬스도 중산층 저택을 의미하니 다른 요소가 숨겨줄것이다.
nuls[nuls >= 0.8]

#여담이지만 앙상블 기법을 이용하거나, 차원 축소기법으로 복수의 데이터를 하나로 바꿔서 입력에 넣을까했지만
#그렇게 되면 실제 서비스할때도 그런 구조로 입력값을 수정해야 하므로 대회에는 적합하지 않다(입력 방법이 정해져 있으니)

In [ ]:
remove= nuls[nuls >= 0.8].keys()
Set = Set.drop(remove, axis=1)
#쓸모없는 데이터는 없애는 습관이 중요하다. 

In [ ]:
removeNullist(remove)

In [ ]:
remove =('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
         'MasVnrType','FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond')
for col in remove:
    Set[col] = Set[col].fillna('None')
removeNullist(remove)    

In [ ]:
#면적이나 갯수 데이터가 없단건....그 시설이 없다고 봐도 무방할거라 생각한다.
remove =('GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
         'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea','LotFrontage')
for col in remove:
    Set[col] = Set[col].fillna(0)
removeNullist(remove)    

In [ ]:
remove =('MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'Electrical', 'KitchenQual', 'Functional', 'SaleType')
for col in remove:
    Set[col] = Set[col].fillna(Set[col].mode()[0])
removeNullist(remove) 
#이것들은 이해가 안돼서 찾아본 결과, 시설 구조물은 평균이란게 있다고 가정한다. 예를 들어 부엌이 없는 곳은 고시촌 아님 없을 거다

In [ ]:
oneHotneeds = Set.select_dtypes(include='object')
numeric = Set.select_dtypes(exclude='object')


In [ ]:
oneHotneeds = pd.get_dummies(oneHotneeds)
for i in oneHotneeds.columns.to_list():
    if "None" in i:
        oneHotneeds.drop(i,axis=1,inplace=True)
#내가 None이란 데이터를 추가한 아이들에겐 없애주는 과정이 필요하다. 

In [ ]:
Set=pd.concat([numeric,oneHotneeds],axis=1)
processedTrain=Set[:trainSetSize]
processedTest=Set[trainSetSize:]

In [ ]:
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.8, random_state=45)
elastic_net.fit(processedTrain, trainAnswer)


In [ ]:
anwser=elastic_net.predict(processedTest)
result=pd.DataFrame({'Id': testSetId.to_list(), 'SalePrice': anwser})

In [ ]:
result.to_csv('submission.csv', index=False)
#결과 자체는 나쁘지 않았으나 역시 전처리를 좀더 하거나 앙상블을 써보는 것도 좋았을 것 같다